# 🎁 Gifty AI Worker: Product Scorer

Этот ноутбук предназначен для запуска на **Kaggle** с использованием **GPU (T4 x2)**.
Он автоматически берет товары из системы Gifty, оценивает их «подарочность» с помощью LLM и отправляет результаты обратно.

### Инструкция:
1. Включите **Accelerator: T4 x2** в настройках (Settings -> Accelerator).
2. Добавьте свои данные в секцию **Configuration** (API_BASE и API_TOKEN).
3. Запустите все ячейки.

In [ ]:
!pip install -q transformers torch accelerate requests

In [ ]:
import requests
import time
import json
import torch
import logging
from transformers import pipeline
from kaggle_secrets import UserSecretsClient

class GiftyInternalClient:
    def __init__(self, api_base, token):
        self.api_base = api_base.rstrip("/")
        self.headers = {"X-Internal-Token": token}
        self.logger = logging.getLogger("GiftyWorker")

    def get_scoring_tasks(self, limit=50):
        url = f"{self.api_base}/internal/scoring/tasks?limit={limit}"
        resp = requests.get(url, headers=self.headers)
        resp.raise_for_status()
        return resp.json()

    def submit_scoring(self, results):
        url = f"{self.api_base}/internal/scoring/submit"
        resp = requests.post(url, json={"results": results}, headers=self.headers)
        resp.raise_for_status()
        return resp.json()

# Помощник для извлечения JSON из ответа ИИ
def extract_json(text):
    try:
        start = text.find('{')
        end = text.rfind('}') + 1
        return json.loads(text[start:end])
    except: 
        return None

## ⚙️ Configuration
Заполни свои данные ниже или используй Kaggle Secrets.

In [ ]:
# Настройки подключения
API_BASE = "https://api.giftyai.ru" # <-- ЗАМЕНИ НА СВОЙ

try:
    user_secrets = UserSecretsClient()
    API_TOKEN = user_secrets.get_secret("INTERNAL_API_TOKEN")
except:
    API_TOKEN = "your_secret_token_here" # <-- ИЛИ ВПИШИ ТУТ

# Настройки модели
MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"
BATCH_SIZE = 10

## 🤖 Model Initialization

In [ ]:
print(f"Loading model {MODEL_NAME}...")
pipe = pipeline(
    "text-generation", 
    model=MODEL_NAME, 
    device_map="auto", 
    model_kwargs={"torch_dtype": torch.bfloat16}
)

## 🏃‍♂️ Worker Loop

In [ ]:
def process_one_item(task):
    prompt = f"""Проанализируй товар и оцени его как подарок.
    Название: {task['title']}
    Категория: {task.get('category')}
    Описание: {task.get('content_text')}
    
    Верни ТОЛЬКО JSON формат:
    {{
        "score": 0.0-10.0, 
        "reasoning": "краткое обоснование на русском языке"
    }}"""
    
    messages = [{"role": "user", "content": prompt}]
    response = pipe(messages, max_new_tokens=256, return_full_text=False)[0]['generated_text']
    
    data = extract_json(response)
    if not data:
        return 5.0, "Ошибка анализа" # fallback
        
    return data.get('score', 5.0), data.get('reasoning', "Без комментариев")

client = GiftyInternalClient(API_BASE, API_TOKEN)
print("Worker started...")

while True:
    try:
        tasks = client.get_scoring_tasks(limit=BATCH_SIZE)
        if not tasks:
            print("No tasks found. Waiting 60s...")
            time.sleep(60)
            continue
            
        print(f"Processing batch of {len(tasks)} items...")
        batch_results = []
        
        for task in tasks:
            score, reason = process_one_item(task)
            batch_results.append({
                "gift_id": task['gift_id'],
                "llm_gift_score": score,
                "llm_gift_reasoning": reason,
                "llm_scoring_model": MODEL_NAME,
                "llm_scoring_version": "v1.0"
            })
            
        client.submit_scoring(batch_results)
        print(f"Successfully submitted results for {len(tasks)} items.")
        
    except Exception as e:
        print(f"Error in loop: {e}")
        time.sleep(10)